In [ ]:
import os
%cd ..
os.getcwd()

In [5]:
import pygame
import importlib
import src.animation
importlib.reload(src.animation)
from src.animation import Animation
from src.utils import load_dict_from_folder

In [16]:
pygame.init()
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
running = True
player_x, player_y = WIDTH // 2, HEIGHT // 2
player_size = 50 # 50 x 50
image_path = os.path.join("images", "backgrounds", "volcano.png")

background = pygame.image.load(image_path)
background = pygame.transform.scale(background, (WIDTH, HEIGHT))


In [7]:
import json
with open("data/input.json") as f:
    combos = json.load(f)
combos

{'movement': {'walk': {'left': ['A'],
   'right': ['D'],
   'up': ['W'],
   'down': ['S']},
  'run': {'left': ['A', 'A'],
   'right': ['D', 'D'],
   'stop_conditions': {'left': ['D'], 'right': ['A']}}},
 'actions': {'attack': {'normal': ['J'],
   'running': {'input': ['J'], 'stop_after_attack': True}},
  'defend': ['K'],
  'jump': ['L']},
 'timings': {'double_tap_window_ms': 300,
  'attack_cooldown_ms': 500,
  'defend_duration_ms': 1000,
  'jump_duration_ms': 400}}

In [11]:
combos['movement']['walk'].clear()

combos['movement']['walk']

{}

In [ ]:
class Player:
    def __init__(self, config):
        self.x = x
        self.y = y
        self.base_speed = speed
        self.animations = animations
        self.state = "idle"
        self.is_running = False
        
    def update(self, events, keys):
        current_time = pygame.time.get_ticks()
        for event in events:
            if event.type == pygame.KEYDOWN:
                if self.is_running:
    
    def draw(self, screen):
        pass

In [ ]:
class InputSystem:
    def __init__(self, player):
        self.player = player
        self.buffer = []
        self.last_input_time = 0
        
    def update(self, events):
        current_time = pygame.time.get_ticks()
        for event in events:
            if event.type == pygame.KEYDOWN:
                key = pygame.key.name(event.key).upper()
                self.buffer.append((key, current_time))
        
        # Clean old inputs
        self.buffer = [
            (k, t) for k, t in self.buffer 
            if current_time - t < self.player.config["timings"]["double_tap_window_ms"]
        ]

In [ ]:
class MovementSystem:
    def __init__(self, player):
        self.player = player
        self.is_running = False
        self.facing_right = True
        self.last_key_press = {} # {'A': '35.45'} for ex
    
    def detect_double_tap(self, key): #this function only works with doutaps of 2 keyssame
        current_time = pygame.time.get_ticks()
        if key in self.last_key_press:
            last_time = self.last_key_press[key]
            if current_time - last_time < 300:
                self.last_key_press.clear()
                print("Detect double tap")
                return True
        self.last_key_press[key] = current_time
        return False
        
    def update(self):
        walk_cfg = self.player.config['movement']['walk']
        run_cfg = self.player.config['movement']['run']
        
        if self.detect_double_tap("D"):
            self.is_running = True
            self.facing_right = True
            
        elif self.detect_double_tap("A"):
            self.is_running = True
            self.facing_right = False
            
        if self.is_running:
            stop_keys = run_cfg["stop_conditions"]["right"] if self.facing_right else run_cfg["stop_conditions"]["left"]
            if any(k in self.player.components['input'].buffer for k in stop_keys):
                self.is_running = False

        

In [22]:
player_dict = load_dict_from_folder("images/characters/mark")
player_dict

{'attack': <Surface(320x80x32 SW)>,
 'defend': <Surface(160x80x32 SW)>,
 'idle': <Surface(320x80x32 SW)>,
 'jump': <Surface(240x80x32 SW)>,
 'move': <Surface(320x80x32 SW)>,
 'run': <Surface(240x80x32 SW)>,
 'run_attack': <Surface(560x80x32 SW)>}

In [24]:
class StateManager:
    STATES = ["idle", "walk", "run", "attack", "defend", "jump"]
    
    def __init__(self):
        self.current = "idle"
        self.previous = None
    
    def change(self, new_state):
        if new_state in self.STATES:
            self.previous = self.current
            self.current = new_state

In [25]:
class Animations:
    def __init__(self):
        self.animations = {
            "idle": Animation(player_dict["idle"], 300),
            "move": Animation(player_dict["move"], 150),
            "attack": Animation(player_dict["attack"], 200),
            "run": Animation(player_dict["run"], 150),
            "defend": Animation(player_dict["defend"], 300),
            "jump": Animation(player_dict["jump"], 200),
            "run_attack": Animation(player_dict["run_attack"], 150)
        }
        
    def update(self, state, facing_right):
        animation = self.animations.get(state.lower(), self.animations["idle"]) # state attack, ATTACK, attACK
        if self.current_animation != animation:
            self.current_animation = animation
            self.current_frame = 0
            self.last_update = pygame.time.get_ticks()
        
        # Advance frame if cooldown passed
        now = pygame.time.get_ticks()
        if now - self.last_update > animation.cooldown:
            self.current_frame = (self.current_frame + 1) % animation.frame_num
            self.last_update = now

    def get_current_frame(self):
        """Returns properly flipped frame based on facing direction"""
        frame = self.current_animation.frames[self.current_frame]
        return pygame.transform.flip(frame, not self.facing_right, False)

In [27]:
class Player:
    def __init__(self, config):
        self.config = config
        self.components = {
            'input': InputSystem(self),
            'movement': MovementSystem(self),
            'animations': Animations(),
            # 'actions': ActionSystem(self),
            'state': StateManager()
        }
    
    def update(self, events):
        self.components['input'].update(events)
        self.components['movement'].update()
        self.components['animation'].update(
            state=self.components['state'].current,
            facing_right=self.components['movement'].facing_right
        )

    def draw(self, screen):
        frame = self.components['animation'].get_current_frame()
        screen.blit(frame, (self.x, self.y))
        
    # def draw(self, screen):
    #     """Draw the player on screen"""
    #     frame = self.animations[self.state].get_current_frame()
    #     if not self.facing_right:
    #         frame = pygame.transform.flip(frame, True, False)
    #     screen.blit(frame, (self.x, self.y))


In [21]:
move = MovementSystem(5)

background = pygame.image.load("images/backgrounds/good_back.jpg")
background = pygame.transform.scale(background, screen.get_size())

running = True
while running:
    # screen.blit(background, (0, 0))  
    keys = pygame.key.get_pressed()
    events = pygame.event.get()
    
    for event in events:
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            move.detect_double_tap(event.key)
    pygame.display.flip()
    clock.tick(60) 


Detect double tap
Detect double tap
Detect double tap
Detect double tap
Detect double tap
Detect double tap


KeyboardInterrupt: 

In [28]:
import pygame
import sys

# Initialize pygame
pygame.init()

# Screen setup
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Fighting Game")
clock = pygame.time.Clock()

# Load assets (example - replace with your actual loading code)
def load_animations():
    return {
        "idle": Animation([pygame.Surface((50, 80)) for _ in range(4)], 300),
        "move": Animation([pygame.Surface((50, 80)) for _ in range(6)], 150),
        "attack": Animation([pygame.Surface((60, 80)) for _ in range(8)], 200),
        "run": Animation([pygame.Surface((55, 80)) for _ in range(6)], 150),
        "defend": Animation([pygame.Surface((50, 80)) for _ in range(3)], 300),
        "jump": Animation([pygame.Surface((50, 80)) for _ in range(5)], 200),
        "run_attack": Animation([pygame.Surface((70, 80)) for _ in range(8)], 150)
    }

# Create player
player_animations = load_animations()
player = Player(x=100, y=300, speed=5, animations=player_animations)

# Main game loop
def main():
    running = True
    
    while running:
        # 1. Process events
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.QUIT:
                running = False
        
        # 2. Update game state
        keys = pygame.key.get_pressed()
        player.update(events)
        
        # 3. Render
        screen.fill((0, 0, 0))  # Clear screen
        
        # Draw background (example)
        pygame.draw.rect(screen, (50, 50, 100), (0, 400, WIDTH, 200))  # "Ground"
        
        # Draw player
        player.draw(screen)
        
        # Debug info (optional)
        font = pygame.font.SysFont(None, 24)
        debug_text = f"State: {player.components['state'].current} | Pos: ({player.x}, {player.y})"
        debug_surface = font.render(debug_text, True, (255, 255, 255))
        screen.blit(debug_surface, (10, 10))
        
        pygame.display.flip()
        clock.tick(60)  # 60 FPS

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

AttributeError: 'list' object has no attribute 'get_width'

In [ ]:
class AnimationComponent:
    def __init__(self, animations):
        self.animations = animations  # Loaded from your `load_dict_from_folder`
        self.current = None
    
    def play(self, name, flip=False):
        if name != self.current:
            self.current = name
            self.frame_index = 0
            self.last_update = 0
        
        # Advance frames based on animation cooldown
        now = pygame.time.get_ticks()
        anim = self.animations[name]
        if now - self.last_update > anim.cooldown:
            self.frame_index = (self.frame_index + 1) % len(anim.frames)
            self.last_update = now
        
        return pygame.transform.flip(anim.frames[self.frame_index], flip, False)

In [ ]:
class InputBuffer:
    def __init__(self):
        self.buffer = []
        self.buffer_duration = 500  # ms
    
    def add(self, input):
        self.buffer.append((input, pygame.time.get_ticks()))
        self._clean()
    
    def _clean(self):
        now = pygame.time.get_ticks()
        self.buffer = [(i,t) for i,t in self.buffer if now - t < self.buffer_duration]
    
    def match_combo(self, sequence):
        return any(
            self.buffer[-len(sequence):] == sequence 
            for sequence in COMBO_LIBRARY
        )

In [ ]:
class Player:
    def __init__(self, config):
        self.components = {
            'input': InputComponent(config['controls']),
            'physics': PhysicsComponent(config['physics']),
            'combat': CombatComponent(config['attacks']),
            'animation': AnimationComponent(config['animations'])
        }
    
    def update(self):
        for component in self.components.values():
            component.update(self)